# **Cars**

Con el objetivo de consolidar los conocimientos adquiridos, nos embarcaremos en la tarea de descargar todos los anuncios de una página de MercadoLibre, en la cual filtraremos los datos de los automóviles que se encuentran en venta en el Estado de México, específicamente en Cuautitlán Izcalli. Nuestra meta principal será la extracción exhaustiva de información relevante, la creación de un DataFrame que albergue los datos recolectados y, además, la obtención de imágenes de cada uno de los vehículos. Todo esto será llevado a cabo aplicando las habilidades previamente adquiridas en el uso de las bibliotecas BeautifulSoup y Requests.

# **Configuraciones**

**Bibliotecas**

In [ ]:
# Importación de las bibliotecas necesarias
import pandas as pd  # Para el manejo de datos en DataFrames
import time  # Para la pausa entre solicitudes
from urllib.request import urlopen, urlretrieve  # Para hacer solicitudes HTTP y descargar imágenes
from bs4 import BeautifulSoup  # Para analizar el contenido HTML

**Drive**

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Vamos a configurar una estructura de directorios en la que crearemos una carpeta principal llamada 'output'. Dentro de esta carpeta 'output', vamos a crear dos subcarpetas adicionales: 'data' e 'img'. La carpeta 'data' se utilizará para almacenar nuestro DataFrame, mientras que la carpeta 'img' será destinada para guardar nuestras imágenes.

# **WebScraping**

Con el siguiente código, obtendremos los datos de nuestro DataFrame

In [ ]:
# Algoritmo para saber el número de páginas usando list comprehension
numeros = [(48 * n) + 1 for n in range(1, 14)]

# URL inicial
url_inicial = "https://autos.mercadolibre.com.mx/estado-de-mexico/cuautitlan-izcalli/autos_NoIndex_True#applied_filter_id%3Dcity%26applied_filter_name%3DCiudades%26applied_filter_order%3D6%26applied_value_id%3DTUxNQ0NVQTMwNDU%26applied_value_name%3DCuautitl%C3%A1n+Izcalli%26applied_value_order%3D23%26applied_value_results%3D652%26is_custom%3Dfalse"

# Lista de URLs que estaremos raspando usando list comprehension
base_url = "https://autos.mercadolibre.com.mx/estado-de-mexico/cuautitlan-izcalli/autos_Desde_"
paginas = [url_inicial] + [f"{base_url}{n}_NoIndex_True" for n in numeros]

# Lista para almacenar cada diccionario con los datos de los carros
cards = []

# Loop para crear el objeto BeautifulSoup en cada URL
for url in paginas:
    response = urlopen(url)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    anuncios = soup.find_all('div', class_="andes-card ui-search-result ui-search-result--mot andes-card--flat andes-card--padding-16 andes-card--animated")

    # Loop para obtener los datos de cada card
    for anuncio in anuncios:
        card = {}
        modelo_elemento = anuncio.find('h2', class_="ui-search-item__title")
        if modelo_elemento:
            card['Modelo'] = modelo_elemento.get_text()
        else:
            card['Modelo'] = 'No se encontró el modelo'

        precio_elemento = anuncio.find('span', class_="andes-money-amount__fraction")
        if precio_elemento:
            card['Precio'] = precio_elemento.get_text()
        else:
            card['Precio'] = 'No se encontró el precio'

        ano_elemento = anuncio.find('li', class_="ui-search-card-attributes__attribute")
        if ano_elemento:
            card['Año'] = ano_elemento.get_text()
        else:
            card['Año'] = 'No se encontró el año'

        km_elemento = anuncio.find_all('li', class_="ui-search-card-attributes__attribute")
        if len(km_elemento) > 1:
            card['Km'] = km_elemento[1].get_text()
        else:
            card['Km'] = 'No se encontraron kilómetros'

        direccion_elemento = anuncio.find('span', class_="ui-search-item__location")
        if direccion_elemento:
            card['Direccion'] = direccion_elemento.get_text()
        else:
            card['Direccion'] = 'No se encontró la dirección'

        cards.append(card)

        imagen = anuncio.find('img')
        if imagen:
            img_url = imagen.get('data-src')
            if img_url:
                image_name = img_url.split('/')[-1]
                urlretrieve(img_url, f'/content/drive/MyDrive/ouput/img/{image_name}')

time.sleep(2)  # Pausa de 2 segundos entre páginas para no sobrecargar el servidor


In [ ]:
cards

[{'Modelo': 'Ford Escape 2.0 Titanium Ecoboost At',
  'Precio': '525,000',
  'Año': '2020',
  'Km': '87,000 Km',
  'Direccion': 'Cuautitl√°n Izcalli'},
 {'Modelo': 'Nissan Sentra 1.8 Exclusive At Cvt',
  'Precio': '420,000',
  'Año': '2021',
  'Km': '25,000 Km',
  'Direccion': 'Cuautitl√°n Izcalli'},
 {'Modelo': 'Mazda Mazda 3 2.5 S Grand Touring At',
  'Precio': '320,000',
  'Año': '2018',
  'Km': '47,000 Km',
  'Direccion': 'Cuautitl√°n Izcalli'},
 {'Modelo': 'Nissan X-trail 2.5 Exclusive 2 Row Cvt',
  'Precio': '415,000',
  'Año': '2018',
  'Km': '55,000 Km',
  'Direccion': 'Cuautitl√°n Izcalli'},
 {'Modelo': 'Mini Cooper 1.5 Seven At',
  'Precio': '370,000',
  'Año': '2018',
  'Km': '63,000 Km',
  'Direccion': 'Cuautitl√°n Izcalli'},
 {'Modelo': 'Mazda Mazda 3 2.5 S Grand Touring At',
  'Precio': '330,000',
  'Año': '2018',
  'Km': '98,000 Km',
  'Direccion': 'Cuautitl√°n Izcalli'},
 {'Modelo': 'Honda City 1.5 Lx Mt',
  'Precio': '254,000',
  'Año': '2018',
  'Km': '101,000 Km',
  

In [ ]:
len(cards)

633

# **DataFrame**

Creando nuestro dataframe con los datos obtenidos

In [ ]:
dataset = pd.DataFrame(cards)

In [ ]:
dataset

,Modelo,Precio,Año,Km,Direccion
0,Ford Escape 2.0 Titanium Ecoboost At,"525,000",2020,"87,000 Km",Cuautitl√°n Izcalli
1,Nissan Sentra 1.8 Exclusive At Cvt,"420,000",2021,"25,000 Km",Cuautitl√°n Izcalli
2,Mazda Mazda 3 2.5 S Grand Touring At,"320,000",2018,"47,000 Km",Cuautitl√°n Izcalli
3,Nissan X-trail 2.5 Exclusive 2 Row Cvt,"415,000",2018,"55,000 Km",Cuautitl√°n Izcalli
4,Mini Cooper 1.5 Seven At,"370,000",2018,"63,000 Km",Cuautitl√°n Izcalli
...,...,...,...,...,...
628,Audi A1 1.4 Ego S-tronic Dsg,"250,000",2015,"62,000 Km",Jardines Del Alba
629,Ford Lobo Limited 6.2,"360,000",2011,"88,000 Km",Cuautitl√°n Izcalli
630,Nissan Tsuru 1.6 Gsi Mt,"128,000",2014,"131,000 Km",Cuautitl√°n Izcalli
631,Chevrolet Aveo 1.5 Ls Tm,"169,000",2020,"46,000 Km",Cuautitl√°n Izcalli


He detectado un pequeño error en nuestra información. En lugar de 'Cuautitl√°n Izcalli', debemos reemplazarlo por 'Cuautitlán Izcalli

In [ ]:
# Reemplazar "√°" por "a" en la columna 'Direccion'
dataset['Direccion'] = dataset['Direccion'].str.replace("√°", "a")

In [ ]:
dataset

,Modelo,Precio,Año,Km,Direccion
0,Ford Escape 2.0 Titanium Ecoboost At,"525,000",2020,"87,000 Km",Cuautitlan Izcalli
1,Nissan Sentra 1.8 Exclusive At Cvt,"420,000",2021,"25,000 Km",Cuautitlan Izcalli
2,Mazda Mazda 3 2.5 S Grand Touring At,"320,000",2018,"47,000 Km",Cuautitlan Izcalli
3,Nissan X-trail 2.5 Exclusive 2 Row Cvt,"415,000",2018,"55,000 Km",Cuautitlan Izcalli
4,Mini Cooper 1.5 Seven At,"370,000",2018,"63,000 Km",Cuautitlan Izcalli
...,...,...,...,...,...
628,Audi A1 1.4 Ego S-tronic Dsg,"250,000",2015,"62,000 Km",Jardines Del Alba
629,Ford Lobo Limited 6.2,"360,000",2011,"88,000 Km",Cuautitlan Izcalli
630,Nissan Tsuru 1.6 Gsi Mt,"128,000",2014,"131,000 Km",Cuautitlan Izcalli
631,Chevrolet Aveo 1.5 Ls Tm,"169,000",2020,"46,000 Km",Cuautitlan Izcalli


Finalmente, guardamos nuestro dataset en nuestra carpeta.

In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo = "/content/drive/MyDrive/ouput/data/datos.csv"

# Guarda el DataFrame en un archivo CSV en la ruta especificada
dataset.to_csv(ruta_archivo, index=False)

# **Conclusiones**

Concluyendo nuestro proyecto, hemos logrado con éxito raspar información de anuncios de automóviles de varias páginas web y almacenar estos datos en un DataFrame. También hemos organizado nuestras imágenes y datos en una estructura de carpetas. Ahora tenemos un valioso banco de datos que contiene información sobre vehículos y una colección de imágenes asociadas a estos anuncios.

**¿Qué podemos hacer con nuestro banco de datos y de imágenes?**

1. **Análisis de Datos:** Podemos realizar análisis de datos en el DataFrame para obtener información útil. Esto incluye estadísticas sobre precios, modelos de vehículos, años, kilometraje y ubicaciones.

2. **Visualización de Datos:** Podemos crear visualizaciones gráficas, como gráficos y diagramas, para representar de manera efectiva los patrones y tendencias en los datos.

3. **Machine Learning:** Podemos utilizar el conjunto de datos para entrenar modelos de aprendizaje automático, como modelos de predicción de precios de vehículos, modelos de clasificación de marcas y modelos de recomendación.

4. **Búsqueda y Filtrado:** Podemos crear una interfaz de búsqueda y filtrado para que los usuarios encuentren vehículos específicos en función de sus preferencias.

5. **Desarrollo de Aplicaciones:** Podemos desarrollar aplicaciones web o móviles que utilicen estos datos para brindar información sobre vehículos y facilitar la búsqueda y compra.

6. **Almacenamiento Seguro:** Es importante asegurarse de que nuestro banco de datos y las imágenes se almacenan de manera segura para proteger la privacidad y seguridad de los usuarios.

7. **Actualización Continua:** Mantener el banco de datos actualizado con nuevos anuncios y datos de vehículos.